<h1>Imports</h1>

In [1]:
import numpy as np
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import tensorflow as tf

In [3]:
import keras
keras.__version__

'3.3.3'

In [4]:
%matplotlib inline

In [5]:
import cv2

In [6]:
# adds a progressbar
from tqdm.notebook import tqdm

# set seaborn graphs to a better style
sns.set(style="ticks")

In [7]:
import os, sklearn

<h1>Loading data</h1>

In [8]:
data_path = r"C:\Users\tisha\Documents\GitHub\adv_python_project\face_age"

In [9]:
from sklearn.model_selection import train_test_split

def load_images(data_path, img_size=(200, 200), test_size=0.15, random_state=42):
    images = []
    labels = []

    for age_folder in os.listdir(data_path):
        age_path = os.path.join(data_path, age_folder)
        if os.path.isdir(age_path):
            for img_name in os.listdir(age_path):
                img_path = os.path.join(age_path, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, img_size)
                    images.append(img)
                    labels.append(int(age_folder))

    images = np.array(images)
    labels = np.array(labels)
    images = images / 255.0

    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=test_size, random_state=random_state)

    return X_train, X_test, y_train, y_test

<h1>Processing data</h1>

In [10]:
X_train, X_test, y_train, y_test = load_images(data_path)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

X_train shape: (8311, 200, 200, 3)
X_test shape: (1467, 200, 200, 3)
y_train shape: (8311,)
y_test shape: (1467,)


<h1>Creating model</h1>

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

In [18]:
model = Sequential()

model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(200, 200, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())

model.add(Dense(128, activation='swish'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='swish'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))

#use mean absolute error insted of accuracy bruh

model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae', 'mape'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 198, 198, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 198, 198, 16)        │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 99, 99, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 99, 99, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 97, 97, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 97, 97, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 48, 48, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 48, 48, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 46, 46, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 46, 46, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 23, 23, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 23, 23, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 21, 21, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 21, 21, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 10, 10, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 10, 10, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 12800)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │       1,638,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 1,753,569 (6.69 MB)

 Trainable params: 1,753,089 (6.69 MB)

 Non-trainable params: 480 (1.88 KB)

In [19]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
# from keras.callbacks import ModelCheckpoint

# stop_early = EarlyStopping(patience = 20, monitor="val_loss")


reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=5, min_lr=0.0005)

model_checkpoint = ModelCheckpoint(monitor='val_loss', mode='min', filepath=r"C:\Users\tisha\Documents\GitHub\adv_python_project\model_checkpoints\age_model.keras")             


<h1>Training the model</h1>

In [16]:
# callbacks=[stop_early], 

n_epochs = 50
b_size = 128

history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=b_size, validation_split=0.2, callbacks=[reduce_lr, model_checkpoint], verbose=1)

test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)

Epoch 1/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 147s 2s/step - loss: 748.5662 - mae: 20.9343 - val_loss: 764.5550 - val_mae: 20.6867 - learning_rate: 0.0010
Epoch 2/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 96s 2s/step - loss: 342.2524 - mae: 13.4387 - val_loss: 569.0566 - val_mae: 20.4929 - learning_rate: 0.0010
Epoch 3/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 94s 2s/step - loss: 287.7287 - mae: 12.3231 - val_loss: 528.2846 - val_mae: 19.2157 - learning_rate: 0.0010
Epoch 4/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step - loss: 245.2601 - mae: 11.0168 - val_loss: 478.7341 - val_mae: 18.5502 - learning_rate: 0.0010
Epoch 5/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step - loss: 236.0764 - mae: 11.0582 - val_loss: 419.8119 - val_mae: 16.8414 - learning_rate: 0.0010
Epoch 6/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 94s 2s/step - loss: 209.0882 - mae: 10.2131 - val_loss: 391.9808 - val_mae: 16.4095 - learning_rate: 0.0010
Epoch 7/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step - loss: 202.4616 - mae: 10.1557 - val_loss: 351.9320 - val_mae: 16

Epoch 46/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - loss: 106.4324 - mae: 7.2252 - val_loss: 252.7951 - val_mae: 11.4130 - learning_rate: 1.0000e-05
Epoch 47/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step - loss: 107.0469 - mae: 7.0969 - val_loss: 249.6857 - val_mae: 11.3273 - learning_rate: 1.0000e-05
Epoch 48/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step - loss: 103.2418 - mae: 7.0325 - val_loss: 260.1357 - val_mae: 11.6258 - learning_rate: 1.0000e-05
Epoch 49/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step - loss: 107.8914 - mae: 6.9955 - val_loss: 255.0603 - val_mae: 11.4888 - learning_rate: 1.0000e-05
Epoch 50/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - loss: 101.2240 - mae: 6.9397 - val_loss: 257.5367 - val_mae: 11.5600 - learning_rate: 1.0000e-05
Epoch 51/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - loss: 105.5916 - mae: 7.2063 - val_loss: 259.9225 - val_mae: 11.6338 - learning_rate: 1.0000e-05
Epoch 52/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - loss: 104.5878 - mae: 7.0882 - val_loss:

52/52 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - loss: 99.7647 - mae: 6.9313 - val_loss: 249.7546 - val_mae: 11.3502 - learning_rate: 1.0000e-05
Epoch 91/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - loss: 105.4068 - mae: 7.0864 - val_loss: 257.1870 - val_mae: 11.5696 - learning_rate: 1.0000e-05
Epoch 92/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - loss: 98.6280 - mae: 6.7874 - val_loss: 260.7144 - val_mae: 11.6672 - learning_rate: 1.0000e-05
Epoch 93/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - loss: 104.1352 - mae: 7.0508 - val_loss: 268.4116 - val_mae: 11.8597 - learning_rate: 1.0000e-05
Epoch 94/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - loss: 104.0643 - mae: 7.0630 - val_loss: 263.6245 - val_mae: 11.7173 - learning_rate: 1.0000e-05
Epoch 95/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - loss: 101.8113 - mae: 6.9033 - val_loss: 268.7097 - val_mae: 11.8618 - learning_rate: 1.0000e-05
Epoch 96/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - loss: 102.8078 - mae: 7.0183 - val_loss: 270.5873 - val

52/52 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - loss: 97.6943 - mae: 6.8235 - val_loss: 268.4347 - val_mae: 11.8576 - learning_rate: 1.0000e-05
Epoch 135/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - loss: 99.6941 - mae: 6.7981 - val_loss: 260.6156 - val_mae: 11.6461 - learning_rate: 1.0000e-05
Epoch 136/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - loss: 98.3769 - mae: 6.7662 - val_loss: 253.6396 - val_mae: 11.4664 - learning_rate: 1.0000e-05
Epoch 137/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step - loss: 101.5971 - mae: 6.9585 - val_loss: 253.4450 - val_mae: 11.4550 - learning_rate: 1.0000e-05
Epoch 138/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - loss: 100.9518 - mae: 6.8189 - val_loss: 259.3795 - val_mae: 11.6150 - learning_rate: 1.0000e-05
Epoch 139/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - loss: 105.0902 - mae: 6.9737 - val_loss: 262.4793 - val_mae: 11.6923 - learning_rate: 1.0000e-05
Epoch 140/200
52/52 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - loss: 96.0695 - mae: 6.7612 - val_loss: 259.7189 -

 7/52 ━━━━━━━━━━━━━━━━━━━━ 1:24 2s/step - loss: 125.3393 - mae: 7.3305

KeyboardInterrupt: 

In [20]:
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

plt.plot(history.epoch, training_loss, 'bo', label='Training Loss')  
plt.plot(history.epoch, validation_loss, 'b', label='Validation Loss')  

plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.legend()
plt.show()

NameError: name 'history' is not defined

In [21]:
print(history.history.keys())

NameError: name 'history' is not defined

In [22]:
mae = history.history['mae']
val_mae = history.history['val_mae']

plt.plot(epochs, mae, 'b', label='Training MAE')
plt.plot(epochs, val_mae, 'r', label='Validation MAE')
plt.title('Training and Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.grid()
plt.show()

NameError: name 'history' is not defined

<h1>Testing the model</h1>

In [23]:
image_path = r"C:\Users\tisha\Documents\GitHub\adv_python_project\test.jpg"
img = cv2.imread(image_path)  
img = cv2.resize(img, (200, 200))  
img = img.astype('float32') / 255.0  


img = np.expand_dims(img, axis=0)

# Check the shape
print('Image shape:', img.shape)

Image shape: (1, 200, 200, 3)


In [24]:
predicted_age = model.predict(img)

print('Predicted age:', predicted_age[0][0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
Predicted age: -0.0074065486


In [ ]:
from keras.models import load_model

model_load_path = r"C:\Users\tisha\Documents\GitHub\adv_python_project\model_checkpoints\age_model.keras"

